In [ ]:
#=======================================================================================
# P-TUNING Q&A=기계독해 훈련 예제
#
# => input_ids : [CLS]질문[SEP]지문[SEP]
# => attention_mask : 1111111111(질문, 지문 모두 1)
# => token_type_ids : 0000000(질문)1111111(지문)
# => start_positions : 45 (질문에 대한 지문에서의 답변 시작 위치)
# => end_positions : 60 (질문에 대한 지문에서의 답변 끝 위치)
#
# prefix-tuning => GPT-2, T5등 LM에서 접두사 prompt를 추가하여 훈련시키는 방식
#
# p-tuning => P-tuning은 prefix-tuning보다 유연 합니다. 
# 시작할 때뿐만 아니라 프롬프트 중간에 학습 가능한 토큰을 삽입하기 때문입니다. 
# https://github.com/THUDM/P-tuning
#
# p-tuing v2 => 새로운 방식이 아니라 NLU 향상을 위해, MLM 모델에 prefix-tuning을 적용한 방식
# https://github.com/THUDM/P-tuning-v2
#=======================================================================================
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
from transformers import AutoConfig, AutoTokenizer, set_seed
import os
from os import sys
from transformers import BertTokenizer, HfArgumentParser, TrainingArguments

sys.path.append('..')
from myutils import seed_everything, GPU_info, mlogging
from tqdm.notebook import tqdm


logger = mlogging(loggername="bert-p-tuing", logfilename="bert-p-tuing")
device = GPU_info()

model_path = '../../data11/model/bert/bert-multilingual-cased'

#tokenize 설정
#tokenizer = AutoTokenizer.from_pretrained(model_path, TOKENIZERS_PARALLELISM=True)
tokenizer = BertTokenizer.from_pretrained(model_path, strip_accents=False, do_lower_case=False)


In [ ]:
# 인자들 설정
#======================================================
# data args 인자 설정 
class data_args:
    dataset_name='QA'
    #pad_to_max_length = True
    max_seq_length = 128
    #overwrite_cache = True
    #max_train_samples = None
    #max_eval_samples = None
    #max_predict_samples = None
    
#======================================================

#======================================================
# 허깅페이스 TrainingArguments 설정
training_args = TrainingArguments("p-tuning-bert-test")

# run_rte_bert.sh 에 사용된 인자만 새롭게 정의함.
training_args.do_train=True
training_args.do_eval=True
training_args.seed = 111
training_args.per_device_train_batch_size=32  #batch_size
training_args.learning_rate =3e-5        #lr
training_args.save_strategy = "no"
training_args.evaluation_strategy = "steps"  #eval 언제마다 할지 => no, steps, epoch
training_args.num_train_epochs = 10 # epochs
training_args.output_dir = '../../data11/model/bert/bert-multilingual-cased-p-tuing-pp-qa-20'

#training_args.report_to = "none"  # 기본은 all로 , all이면 wandb에도 기록된다.
#======================================================
   
#======================================================
# p-tuningv2 prefixt 튜닝일때 설정값 지정 
pre_seq_len = 20             # prefix 계수
prefix_projection = True     # True = two-layer MLP 사용함(Multi-layer perceptron(다중퍼셉트론))
prefix_hidden_size = 512     # prefix hidden size
#======================================================

#======================================================
#seed 설정
#set_seed(training_args.seed)
seed_everything(training_args.seed)
#======================================================

print(training_args.fp16)
print(training_args.get_process_log_level())
print(training_args.do_train)
print(training_args.evaluation_strategy)
print(training_args.learning_rate)

In [ ]:
# 학습 data loader 생성
sys.path.append('..')
from myutils import KorQuADCorpus, QADataset, data_collator
from torch.utils.data import DataLoader, RandomSampler

#############################################################################
# 변수 설정
#############################################################################
max_seq_len = data_args.max_seq_length   # 질문 + 지문 최대 크기
doc_stride = 64     # 지문이 128을 넘을 경우, 얼만큼씩 다음 지문으로 대체할지
max_query_length = 32  # 질문 최대 크기
batch_size = training_args.per_device_train_batch_size        # 배치 사이즈(64면 GUP Memory 오류 나므로, 32 이하로 설정할것=>max_seq_length 를 줄이면, 64도 가능함)
cache = False   # 캐쉬파일 생성할거면 True로 (True이면 loding할때 캐쉬파일있어도 이용안함)
#############################################################################

# corpus 파일 설정
corpus = KorQuADCorpus()


# 학습 dataset 생성
print('create train_loader===========================================================')
train_file_fpath = '../../data11/korpora/korQuAD/KorQuAD_v1.0_train.json'
train_dataset = QADataset(file_fpath=train_file_fpath, tokenizer=tokenizer, corpus=corpus, max_seq_length=max_seq_len, max_query_length = max_query_length, doc_stride= doc_stride, overwrite_cache=cache)


# 학습 dataloader 생성
train_loader = DataLoader(train_dataset, 
                          batch_size=batch_size, 
                          #shuffle=True, # dataset을 섞음
                          sampler=RandomSampler(train_dataset, replacement=False), #dataset을 랜덤하게 샘플링함
                          collate_fn=data_collator, # dataset을 tensor로 변환(예시 {'input_ids':tensor[0,1,2,3,1,], 'token_type_id:tensor[0,0,0,0,0], 'attention_mask:tensor[1,1,1,1,1], 'labels':tensor[5]}
                          num_workers=4)

print('end train_loader===========================================================')

print('create eval_loader===========================================================')
eval_file_fpath = '../../data11/korpora/korQuAD/KorQuAD_v1.0_dev.json'
eval_dataset = QADataset(file_fpath=eval_file_fpath, tokenizer=tokenizer, corpus=corpus, max_seq_length=max_seq_len, max_query_length = max_query_length, doc_stride= doc_stride, overwrite_cache=cache)


# 평가 dataloader 생성
eval_loader = DataLoader(eval_dataset, 
                          batch_size=batch_size, 
                          #shuffle=True, # dataset을 섞음
                          sampler=RandomSampler(eval_dataset, replacement=False), #dataset을 랜덤하게 샘플링함
                          collate_fn=data_collator, # dataset을 tensor로 변환(예시 {'input_ids':tensor[0,1,2,3,1,], 'token_type_id:tensor[0,0,0,0,0], 'attention_mask:tensor[1,1,1,1,1], 'labels':tensor[5]}
                          num_workers=4)
print('end eval_loader===========================================================')

print('train_loader_len: {}, eval_loader_len: {}'.format(len(train_loader), len(eval_loader)))

In [ ]:
# config 설정 
config = AutoConfig.from_pretrained(
    model_path,
    num_labels=2,  # q&a 는 2
    revision="main"
)

#========================================================================
# 훈련 모델에 따라 아래값들을 바꿔줘야 함.
#========================================================================
#get_model 에서 --prefix인경우 config 인자 설정해 주고 있음.

config.hidden_dropout_prob = 0.1
config.pre_seq_len = pre_seq_len             # prefix 계수
config.prefix_projection = prefix_projection    # True = two-layer MLP 사용함(Multi-layer perceptron(다중퍼셉트론))
config.prefix_hidden_size = prefix_hidden_size     # prefix hidden size
#========================================================================

print(config.num_hidden_layers)
print(config.num_attention_heads)
print(config.hidden_size)

In [ ]:
#==============================================================================
# p-tuing v2 => 새로운 방식이 아니라 NLU 향상을 위해, MLM 모델에 prefix-tuning을 적용한 방식
# 참고 소스 : https://github.com/THUDM/P-tuning-v2
# 
# p-tuning-v2의 주요 기여는 원래 입력 전에 사용자 정의 길이의 레이어 프롬프트를 추가하고 
# 다운스트림 작업에 대한 후속 교육에서 BERT 모델의 모든 매개변수를 고정하고 이러한 프롬프트만 교육하는 것임.
# 설명 : https://zhuanlan.zhihu.com/p/459305102
#
# => P-tuning-v2의 구현 방식은 prefix N 시퀀스를 생성한 다음, 원래 bert 모델과 연결한다. 이때 bert의 past_key_values(*여기서는 decoding 속도 개선 목적이 아님)를 이용함
# => bert의 past_key_values로 prefix에대한 key 와 value 를 넘겨줘서, 기존 입력 key, value와 연결시키도록 함.
# => get_prompt() 함수 : prefix를 past_key_value 형식(batch_size, num_heads, sequence_length - 1, embed_size_per_head)으로 조정(만듬)
# => attention_mask : 기존 attention_mask +  prefix_attention_mask 
#==============================================================================

import copy
import torch
import torch.nn
from torch.nn import CrossEntropyLoss
from transformers import BertPreTrainedModel, BertModel
from transformers.modeling_outputs import QuestionAnsweringModelOutput

class PrefixEncoder(torch.nn.Module):
    r'''
    The torch.nn model to encode the prefix
    Input shape: (batch-size, prefix-length)
    Output shape: (batch-size, prefix-length, 2*layers*hidden)
    '''
    def __init__(self, config):
        super().__init__()
        self.prefix_projection = config.prefix_projection
        if self.prefix_projection:
            # Use a two-layer MLP to encode the prefix
            self.embedding = torch.nn.Embedding(config.pre_seq_len, config.hidden_size)
            self.trans = torch.nn.Sequential(
                torch.nn.Linear(config.hidden_size, config.prefix_hidden_size),
                torch.nn.Tanh(),
                
                # num_hidden_layers(12)*2*dim_embedding 인데, 
                # 여기서 *2는 key와 value를 기존 입력 key와 value로 연결시키는 구조이므로, *2를 해준것임.
                torch.nn.Linear(config.prefix_hidden_size, config.num_hidden_layers * 2 * config.hidden_size)
            )
        else:
            self.embedding = torch.nn.Embedding(config.pre_seq_len, config.num_hidden_layers * 2 * config.hidden_size)

    def forward(self, prefix: torch.Tensor):
        if self.prefix_projection:
            prefix_tokens = self.embedding(prefix)
            past_key_values = self.trans(prefix_tokens)
        else:
            past_key_values = self.embedding(prefix)
        return past_key_values
 
# BertPrefixForQuestionAnswering 클래스
class BertPrefixForQuestionAnswering(BertPreTrainedModel):
    def __init__(self, config):
        super().__init__(config)
        self.num_labels = config.num_labels

        self.pre_seq_len = config.pre_seq_len
        self.n_layer = config.num_hidden_layers
        self.n_head = config.num_attention_heads
        self.n_embd = config.hidden_size // config.num_attention_heads

        self.bert = BertModel(config, add_pooling_layer=False)
        self.qa_outputs = torch.nn.Linear(config.hidden_size, config.num_labels)
        self.dropout = torch.nn.Dropout(config.hidden_dropout_prob)
        self.prefix_encoder = PrefixEncoder(config)
        self.prefix_tokens = torch.arange(self.pre_seq_len).long()

        for param in self.bert.parameters():
            param.requires_grad = False

        self.init_weights()

    def get_prompt(self, batch_size):
        prefix_tokens = self.prefix_tokens.unsqueeze(0).expand(batch_size, -1).to(self.bert.device)
        past_key_values = self.prefix_encoder(prefix_tokens)
        bsz, seqlen, _ = past_key_values.shape
        past_key_values = past_key_values.view(
            bsz,
            seqlen,
            self.n_layer * 2, 
            self.n_head,
            self.n_embd
        )
        past_key_values = self.dropout(past_key_values)
        past_key_values = past_key_values.permute([2, 0, 3, 1, 4]).split(2)
        return past_key_values

    def forward(
        self,
        input_ids=None,
        attention_mask=None,
        token_type_ids=None,
        position_ids=None,
        head_mask=None,
        inputs_embeds=None,
        start_positions=None,
        end_positions=None,
        output_attentions=None,
        output_hidden_states=None,
        return_dict=None,
    ):
        r"""
        start_positions (:obj:`torch.LongTensor` of shape :obj:`(batch_size,)`, `optional`):
            Labels for position (index) of the start of the labelled span for computing the token classification loss.
            Positions are clamped to the length of the sequence (:obj:`sequence_length`). Position outside of the
            sequence are not taken into account for computing the loss.
        end_positions (:obj:`torch.LongTensor` of shape :obj:`(batch_size,)`, `optional`):
            Labels for position (index) of the end of the labelled span for computing the token classification loss.
            Positions are clamped to the length of the sequence (:obj:`sequence_length`). Position outside of the
            sequence are not taken into account for computing the loss.
        """
        return_dict = return_dict if return_dict is not None else self.config.use_return_dict

        batch_size = input_ids.shape[0]
        past_key_values = self.get_prompt(batch_size=batch_size)
        prefix_attention_mask = torch.ones(batch_size, self.pre_seq_len).to(self.bert.device)
        attention_mask = torch.cat((prefix_attention_mask, attention_mask), dim=1)

        outputs = self.bert(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            position_ids=position_ids,
            head_mask=head_mask,
            inputs_embeds=inputs_embeds,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
            past_key_values=past_key_values,
        )

        sequence_output = outputs[0]

        logits = self.qa_outputs(sequence_output)
        start_logits, end_logits = logits.split(1, dim=-1)
        start_logits = start_logits.squeeze(-1).contiguous()
        end_logits = end_logits.squeeze(-1).contiguous()

        total_loss = None
        if start_positions is not None and end_positions is not None:
            # If we are on multi-GPU, split add a dimension
            if len(start_positions.size()) > 1:
                start_positions = start_positions.squeeze(-1)
            if len(end_positions.size()) > 1:
                end_positions = end_positions.squeeze(-1)
            # sometimes the start/end positions are outside our model inputs, we ignore these terms
            ignored_index = start_logits.size(1)
            start_positions = start_positions.clamp(0, ignored_index)
            end_positions = end_positions.clamp(0, ignored_index)

            loss_fct = CrossEntropyLoss(ignore_index=ignored_index)
            start_loss = loss_fct(start_logits, start_positions)
            end_loss = loss_fct(end_logits, end_positions)
            total_loss = (start_loss + end_loss) / 2

        if not return_dict:
            output = (start_logits, end_logits) + outputs[2:]
            return ((total_loss,) + output) if total_loss is not None else output

        return QuestionAnsweringModelOutput(
            loss=total_loss,
            start_logits=start_logits,
            end_logits=end_logits,
            hidden_states=outputs.hidden_states,
            attentions=outputs.attentions,
        )

In [ ]:
from transformers import BertForSequenceClassification

# p-tuning 모델
model = BertPrefixForQuestionAnswering.from_pretrained(model_path, config=config, revision="main")

# NLI 모델에서 레벨은 3개지(참,거짓,모름) 이므로, num_labels=3을 입력함
#model = BertForSequenceClassification.from_pretrained(model_path, num_labels=3)

model.to(device)

In [ ]:
# 학습 시작
import time
from transformers import AdamW, get_linear_schedule_with_warmup

logger.info(f"=== model: {model_path} ===")
logger.info(f"num_parameters: {model.num_parameters()}")

##################################################
# 변수 설정
##################################################
epochs = training_args.num_train_epochs            # epochs
learning_rate = training_args.learning_rate  # 학습률
p_itr = 1000           # 손실률 보여줄 step 수
##################################################

# optimizer 적용
optimizer = AdamW(model.parameters(), 
                 lr=learning_rate, 
                 eps=1e-8) # 0으로 나누는 것을 방지하기 위한 epsilon 값(10^-6 ~ 10^-8 사이 이값 입력합)

# 총 훈련과정에서 반복할 스탭
total_steps = len(train_loader)*epochs

num_warmup_steps = total_steps * 0.1

# 스캐줄러 생성
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps=num_warmup_steps, 
                                            num_training_steps=total_steps)

itr = 1
total_loss = 0
total_len = 0
total_correct = 0
list_training_loss = []
list_acc_loss = []
list_validation_acc_loss = []

model.zero_grad()# 그래디언트 초기화
for epoch in tqdm(range(epochs)):

    model.train() # 훈련모드로 변환
    for data in tqdm(train_loader):
    
        #optimizer.zero_grad()
        model.zero_grad()# 그래디언트 초기화
        
        # 입력 값 설정
        input_ids = data['input_ids'].to(device)
        attention_mask = data['attention_mask'].to(device)
        token_type_ids = data['token_type_ids'].to(device)       
        start_positions = data['start_positions'].to(device)
        end_positions = data['end_positions'].to(device)
        
        # 모델 실행
        outputs = model(input_ids=input_ids, 
                        attention_mask=attention_mask,
                        token_type_ids=token_type_ids,
                        start_positions=start_positions,
                        end_positions=end_positions)
        
        # 출력값 loss,logits를 outputs에서 얻어옴
        loss = outputs.loss
        start_scores = outputs.start_logits
        end_scores = outputs.end_logits
        
        # optimizer 과 scheduler 업데이트 시킴
        loss.backward()   # backward 구함
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)   # 그래디언트 클리핑 (gradient vanishing이나 gradient exploding 방지하기 위한 기법)
        optimizer.step()  # 가중치 파라미터 업데이트(optimizer 이동)
        scheduler.step()  # 학습률 감소
        
        # 정확도와 손실률 계산하는 부분은 no_grade 시켜서, 계산량을 줄임.
        # => torch.no_grad()는 gradient을 계산하는 autograd engine를 비활성화 하여 
        # 필요한 메모리를 줄이고, 연산속도를 증가시키는 역활을 함
        with torch.no_grad():
            # 정확도와 총 손실률 계산
            # start 포지션 정확도 구함
            
            # argmax = 최대 인덱스값 리턴함
            start_pred = torch.argmax(F.softmax(start_scores), dim=1)
            start_correct = start_pred.eq(start_positions)

            # end 포지션 정확도 구함
            end_pred = torch.argmax(F.softmax(end_scores), dim=1)
            end_correct = start_pred.eq(end_positions)
            
            # start 포지션과 end 포지션 정확도를 더하고 2로 나줌
            total_correct += (start_correct.sum().item() + end_correct.sum().item()) / 2
                
            total_len += len(start_positions)    
            total_loss += loss.item()
            
            # 주기마다 test(validataion) 데이터로 평가하여 손실류 계산함.
            if itr % p_itr == 0:

                logger.info('[Epoch {}/{}] Iteration {} -> Train Loss: {:.4f}, Train Accuracy: {:.3f}'.format(epoch+1, epochs, itr, total_loss/p_itr, total_correct/total_len))

                list_training_loss.append(total_loss/p_itr)
                list_acc_loss.append(total_correct/total_len)

                total_loss = 0
                total_len = 0
                total_correct = 0

        itr+=1
        
        #if itr > 5:
        #    break
   
    ####################################################################
    # 1epochs 마다 실제 test(validattion)데이터로 평가 해봄
    start = time.time()
    logger.info(f'---------------------------------------------------------')

    # 평가 시작
    model.eval()
    
    total_test_correct = 0
    total_test_len = 0
    
    for data in tqdm(eval_loader):
        # 입력 값 설정
        input_ids = data['input_ids'].to(device)
        attention_mask = data['attention_mask'].to(device)
        token_type_ids = data['token_type_ids'].to(device)       
        start_positions = data['start_positions'].to(device)
        end_positions = data['end_positions'].to(device)
 
        # 손실률 계산하는 부분은 no_grade 시켜서, 계산량을 줄임.
        # => torch.no_grad()는 gradient을 계산하는 autograd engine를 비활성화 하여 
        # 필요한 메모리를 줄이고, 연산속도를 증가시키는 역활을 함
        with torch.no_grad():
            # 모델 실행
            outputs = model(input_ids=input_ids, 
                            attention_mask=attention_mask,
                            token_type_ids=token_type_ids,
                            start_positions=start_positions,
                            end_positions=end_positions)
    
            # 출력값 loss,logits를 outputs에서 얻어옴
            loss = outputs.loss
            start_scores = outputs.start_logits
            end_scores = outputs.end_logits
            
            # 총 손실류 구함
            # start 포지션 정확도 구함
            start_pred = torch.argmax(F.softmax(start_scores), dim=1)
            start_correct = start_pred.eq(start_positions)
            
            # end 포지션 정확도 구함
            end_pred = torch.argmax(F.softmax(end_scores), dim=1)
            end_correct = start_pred.eq(end_positions)
            
            # start 포지션과 end 포지션 정확도를 더하고 2로 나줌
            total_test_correct += (start_correct.sum().item() + end_correct.sum().item()) / 2
            total_test_len += len(start_positions)
    
    list_validation_acc_loss.append(total_test_correct/total_test_len)
    logger.info("[Epoch {}/{}] Validatation Accuracy:{}".format(epoch+1, epochs, total_test_correct / total_test_len))
    logger.info(f'---------------------------------------------------------')
    logger.info(f'=== 처리시간: {time.time() - start:.3f} 초 ===')
    logger.info(f'-END-\n')
    ####################################################################

In [ ]:
# 그래프로 loss 표기
#!pip install matplotlib
import matplotlib.pyplot as plt

plt.plot(list_training_loss, label='Train Loss')
plt.plot(list_acc_loss, label='Train Accuracy')
plt.legend()
plt.show()

In [ ]:
# train loss와 Validatiaon acc 출력
plt.plot(list_training_loss, label='Train Loss')
plt.plot(list_validation_acc_loss, label='Validatiaon Accuracy')
plt.legend()
plt.show()

In [ ]:
### 전체모델 저장
OUTPATH = training_args.output_dir

os.makedirs(OUTPATH, exist_ok=True)
#torch.save(model, OUTPATH + 'pytorch_model.bin') 
model.save_pretrained(OUTPATH)  # save_pretrained 로 저장하면 config.json, pytorch_model.bin 2개의 파일이 생성됨

# tokeinizer 파일 저장(vocab)
VOCAB_PATH = OUTPATH
os.makedirs(VOCAB_PATH,exist_ok=True)
tokenizer.save_pretrained(VOCAB_PATH)

In [ ]:
#torch.save(model, OUTPATH + 'pytorch_model_torch.bin') 